In [1]:
import pandas as pd
import numpy as np
from pyspark.sql import functions as fn
import matplotlib
import matplotlib.pyplot as plt
from pylab import *

In [2]:
df = spark.read.csv('/FileStore/tables/GDA_DATA.csv', header = True)

In [3]:
display(df.limit(200))

###Total number of rides

In [5]:
df.count()

###Total number of subscribers and customers

In [7]:
pd_types = df.groupBy('Subscriber Type').agg(fn.count('*').alias('rides')).toPandas()

pie(pd_types['rides'], labels=pd_types['Subscriber Type'],
                autopct='%1.1f%%', shadow=True, startangle=90)
legend(labels=pd_types['Subscriber Type'])
display(show())

# display(df.groupBy('Subscriber Type').agg(fn.count('*').alias('rides')))

###Total time utilized by subscribers and customers in hours

In [9]:
display(df.groupBy('Subscriber Type').agg(fn.sum('Duration')/3600))

###Average time utilized by subscribers and customers in minutes

In [11]:
display(df.groupBy('Subscriber Type').agg(fn.avg('Duration')/60))

The observation is interesting.. The average ride time utlization time is much more for customers than subscribers. This is true even if the number of subscribers are much more than cuctomers.
There are 4 times more subscribers than customers, but the average ride time utilization of customers is 6 times that of subscribers
In terms of percentage, numeber of subscribers are 394% greater than customers, but average ride time of customers is 577% more than subscribers.

###Recommendation:
It can be seem that customers are lese frequent riders, but usually ride for more time.
An attempt should be made to convert these customers to subscribers.
Fo example, there should be attractive subscription plans for a a customer, who probably rides only on Weekends forlonger time and try to convert himt to subscriber.
The average ride time pf subsccriber should be increased.

###Most popular start stations by numer of rides taken from those stations

In [14]:
display(df.groupBy('Start Station').agg(fn.count('*').alias('rides'), fn.countDistinct('Bike #').alias('bike_count')).orderBy('rides', ascending = False).limit(20))

#####Number of rides and bikes at those staions in decreasing order

In [16]:
display(df.groupBy('End Station').agg(fn.count('*').alias('rides'), fn.countDistinct('Bike #').alias('bike_count')).orderBy('rides', ascending = False))

##Observation
It can be seen that, the San Francisco Cal train station is the most popular station for people to pick bikes from. There are a total of 392 bikes at the station. 
The second most popular station is Harry Bridges Plaza, it also has around same number od bikes.
However, the number of rides at SF Caltrain station is 4500 more than Harry Bridges Plaza.
The bike to ride ratio at SF Caltrain St is 1:33 and at Harry Bridges Plaza is 1:21
For the 10th most popluar station it is 1:13

###Recommendation 1
Address the bike to rides ratio.
If there are 390 bikes for 8300 rides at Harry Bridges Plaza and 386 bikes for 5240 rides at 4th St Market, the number of bikes for SF Caltrain St should be much more than 390 for almost 13,000 rides.

Number of bikes from lesser popluar stations should be transferred to more poplular stations such as SF Caltrain.
This will ensure that more rides are taken from more popluar stations, but the number of rides from lesser poplular stations will not decrease, as an optimum bike to rides ratio will be maintained.

###Recommendation 2
Station such as Temporary Transbay Terminal (Howard at Beale) has many big organizations such as Gap, Twilio, BlackRock, UCB surrounding it, but the utilization of bikes is very less as comapred to stations such as SF Caltrain.
These organizations are very good potential customers. So, there ough to be a marketing strategy to promote bikes are attract these customers. Number of rides should be increased from 5000 odd to 12000. Similar strategy should be applied to similar locations.

###Most popular end stations by number of rides reaching to that station

In [19]:
display(df.groupBy('End Station').agg(fn.count('*').alias('rides')).orderBy('rides', ascending = False))

###Most popular Routes by number of rides taken

In [21]:
display(df.groupBy('Start Station', 'End Station').agg(fn.count('*').alias('rides')).orderBy('rides', ascending = False))

###Which area(zipcode) has the most number of rides

Remove all the rows which have no Zipcode

In [24]:
df.where((fn.col('Zip Code') == 'nil') | (fn.col('Zip Code') == '') | (fn.col('Zip Code') == ' ') | (fn.col('Zip Code') == None)).count()

Data Cleaning: There are 4033 rides with no zipcodes

In [26]:
df_zip = df.where((fn.col('Zip Code') != 'nil') | (fn.col('Zip Code') != '') | (fn.col('Zip Code') != ' ') | (fn.col('Zip Code') != None))

In [27]:
display(df_zip.groupBy('Zip Code').agg(fn.count('*').alias('rides')).orderBy('rides', ascending = False).limit(20))

###Number of rides and hours spent grouped by the area(zipcode)

In [29]:
display(df_zip.groupBy('Zip Code').agg(fn.count('*').alias('rides'), (fn.sum('Duration')/3600).alias('hours')).orderBy('rides', ascending = False).limit(20))

***From above, it is observed that, number of rides at a partiular place corresponds to number of hours. So number of rides at a particular area is directly proportional to the hours spent***

###Relation between number of bikes and number of new users for an area

In [32]:
df_subscriber = df.where(fn.col('Subscriber Type') == 'Subscriber')

In [33]:
display(df_subscriber.groupBy('Zip Code').agg(fn.count('*').alias('no_of_subscriber'), fn.countDistinct('Bike #').alias('no_of_bikes')).orderBy('no_of_bikes', ascending = False).limit(20))

d
###Observation

It can be seen from the above table, that area with zip code 94107 has 15655 subscribers and 542 bikes.
However, area with zip code 94110 has alomost same number of bikes but only 1571 subscribers join from that area.
Thus there is non-uniformity in the distribution of bikes as per the new users. 
For zip code 94110, the bike to new users ratio is 1:3, for every three new users there is 1 bike.
However, for zip code 94107, for every 28 new users there is 1 bike.
###Recommendation
More the subscribers more should be the number of bikes.
There should be a uniformity in the ratio of subscribers and bikes. 
Perhaphs, the bikes from area with zip code 94103 should be moved to an area where people are likely to subscribe more.

##Find which hour is the busiest

In [36]:
from pyspark.sql import types
def get_time(date):
  return int(date.split()[1].split(':')[0])

In [37]:
get_time_udf = fn.udf(get_time, types.IntegerType())
df_start_hour = df.select('*', get_time_udf('Start Date').alias('start_hour'))

In [38]:
display(df_start_hour.groupBy('start_hour').agg(fn.count('*').alias('rides')).orderBy('start_hour', ascending = True))

###Observation:
As 8am, 9am and 5pm, 6pm are the most busiest times, it is evident that office employees are the largest consumer base.
An attempt should be made to attract these customers.
From above observation it is seen that not many people opt for rides around Temporary Transbay Terminal (Howard at Beale) has many big organizations such as Gap, Twilio, BlackRock, UCB surrounding it

##Find which month is the busiest

In [41]:
from pyspark.sql import types
def get_month(date):
  return int(date.split()[0].split('/')[0])

In [42]:
get_month_udf = fn.udf(get_month, types.IntegerType())
df_month = df.select('*', get_month_udf('Start Date').alias('month'))

In [43]:
display(df_month.groupBy('month').agg(fn.count('*').alias('rides')).orderBy('month', ascending = True))

##Observation
There has been a decrease in number of rides from July to August
For san franciso, the avg rainfall decreases from March to May, with June an july being peak sunny periods. This corresponds to the data which shows that bumber of rides increases as rainfall decrases.
RainFall starts increasing from July onwards, which is again depicted in the data as number of rides decreases.
Hence avg rainfall is inversely proprtional to number of rides taken.**

###Number of start and end stations at each zipcode

In [46]:
display(df_zip.groupBy('Zip Code').agg(fn.countDistinct('Start Station').alias('no_of_start_startions'), fn.countDistinct('End Station').alias('no_of_end_startions')).orderBy('no_of_start_startions', ascending = False).limit(20))

####Number of subscribers as per zip code

In [48]:
display(df_subscriber.groupBy('Zip Code').agg(fn.count('*').alias('rides')).orderBy('rides', ascending = False).limit(20))

###Find Lat long of start station and end station

In [50]:
from pyspark.sql import types
Ty = types.ArrayType(types.StructType([types.StructField('start_lat', types.FloatType()),
                                      types.StructField('start_long', types.FloatType()),
                                      types.StructField('end_lat', types.FloatType()),
                                      types.StructField('end_long', types.FloatType())]))

In [51]:
def get_lat_long(start_station, end_station):
  
  ret_list = []
  
  if start_station and end_station:

    response_start = requests.get('https://maps.googleapis.com/maps/api/geocode/json?address=' + start_station.replace(' ', '+') + '+CA,+USA')
  
    response_end = requests.get('https://maps.googleapis.com/maps/api/geocode/json?address=' + end_station.replace(' ', '+') + '+CA,+USA')
    
    if response_start is not None and response_end is not None:
  
      d = {'start_lat': response_start.json()['results'][0]['geometry']['location']['lat'] if response_start.json()['results'][0]['geometry']['location']['lat'] else None,
       'start_long': response_start.json()['results'][0]['geometry']['location']['lng'] if response_start.json()['results'][0]['geometry']['location']['lng'] else None,
       'end_lat': response_end.json()['results'][0]['geometry']['location']['lat'] if response_end.json()['results'][0]['geometry']['location']['lat'] else None,
       'end_long': response_end.json()['results'][0]['geometry']['location']['lng'] if response_end.json()['results'][0]['geometry']['location']['lng'] else None}
  
    ret_list.append(d)
    return ret_list

In [52]:
get_lat_long_udf = fn.udf(get_lat_long, Ty)
df_ll = df.select('*', get_lat_long_udf('Start Station', 'End Station').alias('lat_long'))
df_test = df_ll.select('*', fn.explode('lat_long').alias('lat_lng')).select('*', 'lat_lng.*')

**df_test dataframe contains latitudes and longitudes of start and end stations**

###Group Days of week according to the rides

In [55]:
df_days = spark.read.csv('/FileStore/tables/day_rides.csv', header = True)

In [56]:
display(df_days.groupBy('day_of_week').agg(fn.count('*').alias('rides')))

###Group rides by date and time

In [58]:
def get_date(start_date):
  return str(start_date.split()[0])

In [59]:
df_p = df.toPandas()

In [60]:
df_p.columns

In [61]:
df_p['start_date'] = pd.to_datetime(df_p['Start Date'])
df_p = df_p.sort('start_date')

In [62]:
df_date = sqlContext.createDataFrame(df_p)

In [63]:
df_dates_p = (df_date.groupBy('start_date').agg(fn.count('*').alias('rides'))).toPandas()

In [64]:
df_dates_p['start_date'] = pd.to_datetime(df_dates_p.start_date)
df_dates_p = df_dates_p.sort('start_date')
df_dates_p['start_date'] = df_dates_p['start_date'].astype(str)
df_dates = sqlContext.createDataFrame(df_dates_p)

#####Following graph contains the rides taken at every unique time

In [66]:
display(df_dates)

##Findings and Analysis by City Names

In [68]:
from uszipcode import ZipcodeSearchEngine
df = df.dropna() 
search = ZipcodeSearchEngine()

In [69]:
def zco(x):
    if pd.isnull(x) or x == 'nil' or x == 'Nil':
        return None
    else:
        if '-' in x:
            city = search.by_zipcode((x.split('-')[0]))['City']
        else:            
            city = search.by_zipcode((x))['City']
            if city == NoneType:
                return None
        return city

In [70]:
df['City'] = df['Zip Code'].apply(zco)

The csv files with cities was put back on the cluster server

In [72]:
df_city = spark.read.csv('/FileStore/tables/rides_city.csv', header = True)

In [73]:
df_city = df_city.where(fn.col('City') != 'null')

####Distribution of Rides as per City

In [75]:
display(df_city.groupby('City').agg(fn.count('*').alias('rides')).orderBy('rides', ascending = False))

In [76]:
df_city_p = df_city.toPandas()

In [77]:
def get_date(start_date):
  return str(start_date.split()[0])

In [78]:
df_city_p['start_date'] = df_city_p['Start Date'].apply(get_date)

In [79]:
df_city_p['start_date'] = pd.to_datetime(df_city_p['start_date'])
df_city_p = df_city_p.sort('start_date')

In [80]:
df_date_city = sqlContext.createDataFrame(df_city_p)

####Customer and Subscriber rides distribution in San Francisco

In [82]:
df_SF_sub = df_date_city.where(fn.col('City') == 'San Francisco').where(fn.col('Subscriber Type') == 'Subscriber')

In [83]:
df_SF_sub_p = (df_SF_sub.groupBy('start_date').agg(fn.count('*').alias('rides'))).toPandas()
df_SF_sub_p['start_date'] = pd.to_datetime(df_SF_sub_p.start_date)
df_SF_sub_p = df_SF_sub_p.sort('start_date')
df_SF_sub_p['start_date'] = df_SF_sub_p['start_date'].astype(str)
df_SF_sub_date = sqlContext.createDataFrame(df_SF_sub_p)

#####Subscriber rides distribution in San Francisco

In [85]:
display(df_SF_sub_date)

In [86]:
df_SF_cus = df_date_city.where(fn.col('City') == 'San Francisco').where(fn.col('Subscriber Type') == 'Customer')
df_SF_cus_p = (df_SF_cus.groupBy('start_date').agg(fn.count('*').alias('rides'))).toPandas()
df_SF_cus_p['start_date'] = pd.to_datetime(df_SF_cus_p.start_date)
df_SF_cus_p = df_SF_cus_p.sort('start_date')
df_SF_cus_p['start_date'] = df_SF_cus_p['start_date'].astype(str)
df_SF_cus_date = sqlContext.createDataFrame(df_SF_cus_p)

#####Customer rides distribution in San Francisco

In [88]:
display(df_SF_cus_date)

####Subscriber and Customer rides distribution in San Jose

In [90]:
df_SJ_sub = df_date_city.where(fn.col('City') == 'San Jose').where(fn.col('Subscriber Type') == 'Subscriber')
df_SJ_sub_p = (df_SJ_sub.groupBy('start_date').agg(fn.count('*').alias('rides'))).toPandas()
df_SJ_sub_p['start_date'] = pd.to_datetime(df_SJ_sub_p.start_date)
df_SJ_sub_p = df_SJ_sub_p.sort('start_date')
df_SJ_sub_p['start_date'] = df_SJ_sub_p['start_date'].astype(str)
df_SJ_sub_date = sqlContext.createDataFrame(df_SJ_sub_p)

In [91]:
df_SJ_cus = df_date_city.where(fn.col('City') == 'San Jose').where(fn.col('Subscriber Type') == 'Customer')
df_SJ_cus_p = (df_SJ_cus.groupBy('start_date').agg(fn.count('*').alias('rides'))).toPandas()
df_SJ_cus_p['start_date'] = pd.to_datetime(df_SJ_cus_p.start_date)
df_SJ_cus_p = df_SJ_cus_p.sort('start_date')
df_SJ_cus_p['start_date'] = df_SJ_cus_p['start_date'].astype(str)
df_SJ_cus_date = sqlContext.createDataFrame(df_SJ_cus_p)

In [92]:
df_SJ_cus_date = df_SJ_cus_date.withColumnRenamed('rides', 'rides_cus')
df_SJ_rides = df_SJ_cus_date.join(df_SJ_sub_date, df_SJ_cus_date.start_date == df_SJ_sub_date.start_date).select(df_SJ_cus_date.start_date, df_SJ_cus_date.rides_cus, df_SJ_sub_date.rides)
df_SJ_rides_p = df_SJ_rides.toPandas()
df_SJ_rides_p['start_date'] = pd.to_datetime(df_SJ_rides_p.start_date)
df_SJ_rides_p = df_SJ_rides_p.sort('start_date')
df_SJ_rides_p['start_date'] = df_SJ_rides_p['start_date'].astype(str)
df_SJ_rides_sp = sqlContext.createDataFrame(df_SJ_rides_p)

####Subscriber vs Customer rides distribution in San Jose

In [94]:
display(df_SJ_rides_sp)

####Subscriber vs Customer rides distribution in Oakland

In [96]:
df_O_sub = df_date_city.where(fn.col('City') == 'Oakland').where(fn.col('Subscriber Type') == 'Subscriber')
df_O_sub_date = df_O_sub.groupBy('start_date').agg(fn.count('*').alias('rides_sub'))
df_O_cus = df_date_city.where(fn.col('City') == 'Oakland').where(fn.col('Subscriber Type') == 'Customer')
df_O_cus_date = df_SJ_cus.groupBy('start_date').agg(fn.count('*').alias('rides_cus'))
df_O_rides = df_O_cus_date.join(df_O_sub_date, df_O_cus_date.start_date == df_O_sub_date.start_date).select(df_O_cus_date.start_date, df_O_cus_date.rides_cus, df_O_sub_date.rides_sub)

df_O_rides_p = df_O_rides.toPandas()
df_O_rides_p['start_date'] = pd.to_datetime(df_O_rides_p.start_date)
df_O_rides_p = df_O_rides_p.sort('start_date')
df_O_rides_p['start_date'] = df_O_rides_p['start_date'].astype(str)
df_O_rides_sp = sqlContext.createDataFrame(df_O_rides_p)

In [97]:
display(df_O_rides_sp)